<a href="https://colab.research.google.com/github/cobusgreyling/OpenAI/blob/main/OpenAI_Assistant_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
!pip show openai | grep Version

Version: 1.3.7


In [24]:
import json
import os

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

os.environ['OPENAI_API_KEY'] = str("Your OpenAI API Key goes here.")

In [25]:
from openai import OpenAI

client = OpenAI()



In [26]:
# You can also create Assistants directly through the Assistants API

from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="History Tutor",
    instructions="You are a personal history tutor. Answer questions briefly, in three sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
 'created_at': 1702009585,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal history tutor. Answer questions briefly, in three sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'History Tutor',
 'object': 'assistant',
 'tools': []}

In [27]:
# Creating a new thread:

thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_1flknQB4C8KH4BDYPWsyl0no',
 'created_at': 1702009588,
 'metadata': {},
 'object': 'thread'}

In [28]:
# Now we add a message to the thread:

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What year was the USA founded?",
)
show_json(message)

{'id': 'msg_5xOq4FV38cS98ohBpQPbpUiE',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'What year was the USA founded?'},
   'type': 'text'}],
 'created_at': 1702009591,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no'}

In [29]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_PnwSECkqDDdjWkQ5P7Hcyfor',
 'assistant_id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702009598,
 'expires_at': 1702010198,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal history tutor. Answer questions briefly, in three sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no',
 'tools': []}

In [30]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [31]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_PnwSECkqDDdjWkQ5P7Hcyfor',
 'assistant_id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
 'cancelled_at': None,
 'completed_at': 1702009605,
 'created_at': 1702009598,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal history tutor. Answer questions briefly, in three sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1702009598,
 'status': 'completed',
 'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no',
 'tools': []}

In [32]:
# Now that the Run has completed, list the Messages in the Thread to see what got added by the Assistant.

messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_WhzkHcPnszsmbdrn0H5Ugl7I',
   'assistant_id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
   'content': [{'text': {'annotations': [],
      'value': 'The United States of America was founded in 1776, with the adoption of the Declaration of Independence on July 4th of that year.'},
     'type': 'text'}],
   'created_at': 1702009604,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_PnwSECkqDDdjWkQ5P7Hcyfor',
   'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no'},
  {'id': 'msg_5xOq4FV38cS98ohBpQPbpUiE',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'What year was the USA founded?'},
     'type': 'text'}],
   'created_at': 1702009591,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no'}],
 'object': 'list',
 'first_id': 'msg_WhzkHcPnszsmbdrn0H5Ugl7I',
 'last_id': 'msg_5xOq4FV38cS9

In [33]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you give me a little more detail on this?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_oIOfuARjk20zZRn6lAytf0Hz',
   'assistant_id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! The founding of the USA is marked by the Declaration of Independence, which was ratified by the Continental Congress on July 4, 1776. This act declared the thirteen American colonies free and independent states, breaking away from British rule.'},
     'type': 'text'}],
   'created_at': 1702009645,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_9dWR1QFrN983q1AG1cjcQ9Le',
   'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no'}],
 'object': 'list',
 'first_id': 'msg_oIOfuARjk20zZRn6lAytf0Hz',
 'last_id': 'msg_oIOfuARjk20zZRn6lAytf0Hz',
 'has_more': False}

In [34]:
# Now that the Run has completed, list the Messages in the Thread to see what got added by the Assistant.

messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_oIOfuARjk20zZRn6lAytf0Hz',
   'assistant_id': 'asst_qlaTYRSyl9EWeftjKSskdaco',
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! The founding of the USA is marked by the Declaration of Independence, which was ratified by the Continental Congress on July 4, 1776. This act declared the thirteen American colonies free and independent states, breaking away from British rule.'},
     'type': 'text'}],
   'created_at': 1702009645,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_9dWR1QFrN983q1AG1cjcQ9Le',
   'thread_id': 'thread_1flknQB4C8KH4BDYPWsyl0no'},
  {'id': 'msg_dDeGGSj4w3CIVRd5hsQpGHmF',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'Could you give me a little more detail on this?'},
     'type': 'text'}],
   'created_at': 1702009643,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
  